# MxlPy & MxlBricks workshop
<p align="center">
<img src="https://raw.githubusercontent.com/Computational-Biology-Aachen/MxlPy/refs/heads/main/docs/assets/logo-diagram.png" style="width: 350px; max-width: 45%" alt='mxlpy-logo'>
<img src="https://raw.githubusercontent.com/Computational-Biology-Aachen/mxl-bricks/refs/heads/main/docs/assets/logo.png" style="width: 350px; max-width: 45%" alt='mxlbricks-logo'>
</p>



In [ ]:
from dataclasses import dataclass
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown
from matplotlib.figure import Figure
from mxlpy import (
    Derived,
    Model,
    Simulator,
    compare,
    fns,
    plot,
    report,
    surrogates,
    unwrap,
)

from models import get_sir, get_sird

(TMP_DIR := Path("temp")).mkdir(exist_ok=True, parents=True)


def plot_difference(r_old: pd.DataFrame, r_new: pd.DataFrame) -> Figure:
    rel_diff = (r_new - r_old) / r_old
    largest_diff = rel_diff.abs().mean().fillna(0).sort_values().tail(n=3)

    fig, ax = plot.one_axes()
    plot.lines(r_new, ax=ax)
    lines = dict(zip(r_new.columns, ax.lines, strict=True))
    for f, i in enumerate(reversed(largest_diff.index), start=2):
        line = lines[i]  # type: ignore
        line.set_linewidth(line.get_linewidth() * f)

    plot.reset_prop_cycle(ax)
    plot.lines(r_old, ax=ax, alpha=0.25, legend=False)
    ax.set(xlabel="Time / a.u.", ylabel="Relative Population")
    return fig


def remove_labels(axs: plot.Axs) -> None:
    for ax in axs:
        ax.set(xlabel="", ylabel="")


def grid_labels(
    axs: plot.Axs,
    xlabel: str | None = None,
    ylabel: str | None = None,
) -> None:
    """Apply labels to left and bottom axes."""
    remove_labels(axs)

    col = 0
    invisible = False

    for ax in axs[:, 0]:
        ax.set_ylabel(ylabel)
    for i, ax in enumerate(axs[-1, :]):
        if not ax.get_visible():
            col = i
            invisible = True
            break
        ax.set_xlabel(xlabel)
    if invisible:
        for ax in axs[-2, col:]:
            ax.set_xlabel(xlabel)


## All things derived

Loads of values can be derived from each other.  


### Parameters

In [ ]:
(
    Model()
    .add_parameter("p1", 1.0)
    .add_derived("d1", fns.twice, args=["p1"])  # derive from parameter p1
    .get_dependent()
)

### Variables

In [ ]:
(
    Model()
    .add_variable("v1", 1.0)
    .add_derived("d1", fns.twice, args=["v1"])  # derive from variable v1
    .get_dependent()
)

### Derived variables

In [ ]:
(
    Model()
    .add_parameter("p1", 1.0)
    .add_derived("d1", fns.twice, args=["p1"])
    .add_derived("d2", fns.twice, args=["d1"])  # derive from derived d1
    .get_dependent()
)

### Rates

> Note: does **not** include the stoichiometry, just the rate

In [ ]:
(
    Model()
    .add_variable("v1", 1.0)
    .add_reaction("r1", fns.twice, args=["v1"], stoichiometry={"v1": -1})
    .add_derived("d1", fns.twice, args=["r1"])  # derived from rate of r1
    .add_reaction("r2", fns.twice, args=["d1"], stoichiometry={"v1": -1})  # use d1!
    .get_dependent()
)

### Stoichiometries

Derive stoichiometry from other model components

In [ ]:
(
    Model()
    .add_parameter("p1", 1.0)
    .add_variable("v1", 1.0)
    .add_reaction(
        "r1",
        fns.twice,
        args=["v1"],
        stoichiometry={"v1": Derived(fn=fns.twice, args=["p1"])},
    )
    .get_stoichiometries()
)

### Initial conditions

> Note: this just derives the value **once**.  
> This is **not** the same as a derived variable

In [ ]:
(
    Model()
    .add_variables(
        {
            "v1": 1.0,
            "v2": Derived(fn=fns.twice, args=["v1"]),  # derive initial condition
        }
    )
    .get_initial_conditions()
)

## Surrogates

What **is** a surrogate?  

I will define it here as a replacement / approximation for another system / model.  
These *might* be learned from data, but don't necessarily need to.  


Examples of surrogates

- quasi-steady-states
- polynomials
- machine-learning models (e.g. torch)

You need to check the **validity** of doing these replacements yourself.  
One common criterium would be that the surrogated system is working at a much faster time scale.  
Then one can assume it to be in steady-state instantaneously relative to the model time.  

Surrogates in `MxlPy` can have

- one or multiple inputs
- one or multiple outputs
- one or multiple stoichiometries (factor x output)

### Quasi-steady-state

In [ ]:
def distribute(s: float) -> tuple[float, float]:
    return s / 3, s * 2 / 3


# This creates two derived variables, but has no stoichiometries
(
    Model()
    .add_variables({"a": 1.0})
    .add_surrogate(
        "distribute",
        surrogates.qss.Surrogate(
            model=distribute,
            args=["a"],
            outputs=["a1", "a2"],
        ),
    )
    .get_dependent()
)

### Polynomial

In [ ]:
from numpy.polynomial.polynomial import Polynomial

(
    Model()
    .add_variable("x", 1.0)
    .add_surrogate(
        "surrogate",
        surrogates.poly.Surrogate(
            model=Polynomial(coef=[2]),
            args=["x"],
            outputs=["y"],
        ),
    )
    .add_derived("z", fns.add, args=["x", "y"])
    .get_dependent()
)

### Build your own!

In [ ]:
from mxlpy.types import AbstractSurrogate


@dataclass(kw_only=True)
class DoubleSurrogate(AbstractSurrogate):
    def predict(
        self, args: dict[str, float | pd.Series | pd.DataFrame]
    ) -> dict[str, float]:
        return dict(
            zip(
                self.outputs,
                (args[arg] * 2 for arg in self.args),
                strict=True,
            )
        )  # type: ignore


(
    Model()
    .add_variable("v1", 1.0)
    .add_surrogate(
        "surrogate",
        DoubleSurrogate(
            args=["v1"],
            outputs=["s1"],
        ),
    )
    .get_dependent()
)

## Data references

In [ ]:
def average(light: pd.Series) -> float:
    return light.mean()


lights = pd.Series(
    data={"400nm": 200, "500nm": 300, "600nm": 400},
    dtype=float,
)


(
    Model()
    .add_data("light", lights)
    .add_derived("average_light", average, args=["light"])
    .get_dependent()
)

## Comparisons

In [ ]:
ssc = compare.steady_states(
    get_sir(),
    get_sird(),
)

In [ ]:
_ = ssc.plot_variables()
_ = ssc.plot_fluxes()

In [ ]:
pc = compare.time_courses(
    get_sir(),
    get_sird(),
    time_points=np.linspace(0, 100, 101),
)

_ = pc.plot_variables_relative_difference()
_ = pc.plot_fluxes_relative_difference()

## Reports

In [ ]:
md = report.markdown(
    get_sir(),
    get_sird(),
)

# IPython Display
Markdown(md)

In [ ]:
def analyse_concentrations(m1: Model, m2: Model, img_dir: Path) -> tuple[str, Path]:
    r_old = unwrap(Simulator(m1).simulate(100).get_result())
    r_new = unwrap(Simulator(m2).simulate(100).get_result())
    fig = plot_difference(r_old.variables, r_new.variables)
    fig.savefig((path := img_dir / "concentration.png"), dpi=300)
    plt.close(fig)
    return "## Comparison of largest changing", path


md = report.markdown(
    get_sir(),
    get_sird(),
    analyses=[analyse_concentrations],
    img_path=TMP_DIR,
)

# IPython Display
Markdown(md)

## Metaprogramming

In [ ]:
from mxlpy.meta import (
    generate_latex_code,
    generate_model_code_py,
)


In [ ]:
print(generate_model_code_py(get_sir()))

In [ ]:
print(generate_latex_code(get_sir()))

# Discussion: units

See https://github.com/Computational-Biology-Aachen/MxlPy/issues/26

# MxlBricks

- [Repo](https://github.com/Computational-Biology-Aachen/mxl-bricks)
- [Documentation](https://computational-biology-aachen.github.io/mxl-bricks/0.2.0/)

In [ ]:
from mxlbricks import names as n
from mxlbricks.enzymes import (
    add_catalase,
    add_glycine_decarboxylase_yokota,
    add_glycine_transaminase_yokota,
    add_glycolate_oxidase_yokota,
    add_hpa_outflux,
    add_phosphoglycolate_influx,
    add_serine_glyoxylate_transaminase_irreversible,
)
from mxlbricks.utils import static


def get_yokota1985() -> Model:
    model = Model()
    model.add_variables(
        {
            n.glycolate(): 0.09,
            n.glyoxylate(): 0.7964601770483386,
            n.glycine(): 8.999999999424611,
            n.serine(): 2.5385608670239126,
            n.hydroxypyruvate(): 0.009782608695111009,
            n.h2o2(): 0.010880542843616855,
        }
    )

    add_phosphoglycolate_influx(model)
    add_glycolate_oxidase_yokota(model)
    add_glycine_transaminase_yokota(model)
    add_glycine_decarboxylase_yokota(
        model,
        e0=static(model, n.e0(n.glycine_decarboxylase()), 0.5),
    )
    add_serine_glyoxylate_transaminase_irreversible(model)
    add_hpa_outflux(model)
    add_catalase(model)
    return model

## Create new model that actually has oxygen concentration

In [ ]:
from mxlbricks.enzymes import add_glycolate_oxidase


def get_photorespiration() -> Model:
    model = Model()
    model.add_variables(
        {
            n.glycolate(): 0.09,
            n.glyoxylate(): 0.7964601770483386,
            n.glycine(): 8.999999999424611,
            n.serine(): 2.5385608670239126,
            n.hydroxypyruvate(): 0.009782608695111009,
            n.h2o2(): 0.010880542843616855,
        }
    )
    model.add_parameter(n.o2(), 0.2)  # changed here

    add_phosphoglycolate_influx(model)
    add_glycolate_oxidase(model)  # changed here
    add_glycine_transaminase_yokota(model)
    add_glycine_decarboxylase_yokota(
        model,
        e0=static(model, n.e0(n.glycine_decarboxylase()), 0.5),
    )
    add_serine_glyoxylate_transaminase_irreversible(model)
    add_hpa_outflux(model)
    add_catalase(model)
    return model

In [ ]:
d = get_photorespiration().get_dependent()

In [ ]:
Markdown(
    report.markdown(
        get_yokota1985(),
        get_photorespiration(),
    )
)

In [ ]:
tcc = compare.time_courses(
    get_yokota1985(),
    get_photorespiration(),
    time_points=np.linspace(0, 10, 101, dtype=float),
)

fig, axs = tcc.plot_variables_relative_difference()
remove_labels(axs)
grid_labels(axs, xlabel="Time / h", ylabel="Relative difference")
plt.show()

## Discussion: those are just mechanistic bricks, where is the `L`?  

- Should we provide learned surrogates of these models in a package like this?  
- They can get very large, where do we store them?
  - Downloader like `torch` does it? 